In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("TSM")
qcom = yf.Ticker("QCOM")

tsm_data = tsm.history(period="30d", interval="30m")
qcom_data = qcom.history(period="30d", interval="30m")

In [3]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [4]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-03-28 09:30:00-04:00  135.679993  136.740005  135.440002  135.985001   
2024-03-28 10:00:00-04:00  135.990005  136.330002  135.509995  135.669998   
2024-03-28 10:30:00-04:00  135.710007  136.520004  135.589996  136.259995   
2024-03-28 11:00:00-04:00  136.250000  136.580002  136.210007  136.529999   
2024-03-28 11:30:00-04:00  136.539993  136.800003  136.375000  136.460007   
...                               ...         ...         ...         ...   
2024-05-08 15:30:00-04:00  142.740005  143.679993  142.740005  143.619995   
2024-05-09 09:30:00-04:00  142.940002  143.350006  141.130096  141.470001   
2024-05-09 10:00:00-04:00  141.500000  141.960007  141.149994  141.729996   
2024-05-09 10:30:00-04:00  141.740005  142.380005  141.740005  142.279999   
2024-05-09 11:00:00-04:00  142.279999  142.490005  142.110001  142.139999   

In [5]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [6]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    354
 1.0     13
-1.0     13
Name: count, dtype: int64


In [7]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-380.88340759277344


In [8]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

0.8044546025281172

In [9]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [10]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2024-03-28 09:30:00-04:00   -3.896484e-07
2024-03-28 10:00:00-04:00    2.481132e-01
2024-03-28 10:30:00-04:00    7.174345e-01
2024-03-28 11:00:00-04:00    5.771708e-01
2024-03-28 11:30:00-04:00    5.474037e-01
                                 ...     
2024-05-08 15:30:00-04:00   -1.608193e+00
2024-05-09 09:30:00-04:00   -3.629472e+00
2024-05-09 10:00:00-04:00   -3.160323e+00
2024-05-09 10:30:00-04:00   -3.461677e+00
2024-05-09 11:00:00-04:00   -3.434111e+00
Length: 381, dtype: float64

In [11]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [12]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [13]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [14]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

In [15]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], 1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1]
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = -tsm_qcom_dif['signal']
qcom_data['signal'] = tsm_qcom_dif['signal'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    369
 1.0      6
-1.0      5
Name: count, dtype: int64


TSMC Profit =>  -147.35220336914062
QCOM Profit =>  146.56203900000003
Pair profit =>  -0.790164369140598


In [16]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [17]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']

In [18]:
# calculate cointegration between the two tickers
from statsmodels.tsa.stattools import coint

cointegration = coint(tsm_data['Close'], qcom_data['Close_NORM'])
print(cointegration)



(-2.6380230897566395, 0.22265617381089842, array([-3.92549297, -3.35225646, -3.05562989]))


In [19]:
#Testar Métodos de Mercados Diferentes, Mercados Tecnológicos
# A cointegração muda com 